<a href="https://colab.research.google.com/github/spencer18001/machine-learning-zoomcamp/blob/main/05/hw_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!wget https://github.com/alexeygrigorev/datasets/raw/refs/heads/master/bank-full.csv

In [2]:
%%capture
!pip install pipenv

In [3]:
!pipenv --version #Q1

pipenv, version 2024.2.0


In [4]:
%%capture
!pipenv install scikit-learn==1.5.2

In [5]:
!grep -A 5 'scikit-learn' Pipfile.lock # Q2

        "scikit-learn": {
            "hashes": [
                "sha256:03b6158efa3faaf1feea3faa884c840ebd61b6484167c711548fce208ea09445",
                "sha256:178ddd0a5cb0044464fc1bfc4cca5b1833bfc7bb022d70b05db8530da4bb3dd3",
                "sha256:1ff45e26928d3b4eb767a8f14a9a6efbf1cbff7c05d1fb0f95f211a89fd4f5de",
                "sha256:299406827fb9a4f862626d0fe6c122f5f87f8910b86fe5daa4c32dcd742139b6",


In [6]:
%%capture
PREFIX="https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2024/05-deployment/homework"
!wget $PREFIX/model1.bin
!wget $PREFIX/dv.bin

In [7]:
import pickle

with open('model1.bin', 'rb') as f_in:
    model = pickle.load(f_in)

with open('dv.bin', 'rb') as f_in:
    dv = pickle.load(f_in)

In [8]:
customer = {"job": "management", "duration": 400, "poutcome": "success"}

In [9]:
X = dv.transform([customer])
model.predict_proba(X)[0, 1] # Q3

0.7590966516879658

In [10]:
%%capture
!pipenv install Flask gunicorn

In [11]:
%%capture
!wget -O predict.py https://raw.githubusercontent.com/spencer18001/machine-learning-zoomcamp/main/05/hw/predict.py

In [13]:
import subprocess

process = subprocess.Popen(['pipenv', 'run', 'gunicorn', '--bind', '0.0.0.0:9696', 'predict:app'])
process.pid

2223

In [14]:
import requests

url = "http://0.0.0.0:9696/predict"
client = {"job": "student", "duration": 280, "poutcome": "failure"}
requests.post(url, json=client).json() # Q4

{'subscription': False, 'subscription_probability': 0.33480703475511053}

***This cell is running in the GitHub Codespace environment.***

```shell
docker pull svizor/zoomcamp-model:3.11.5-slim
docker images svizor/zoomcamp-model:3.11.5-slim --format "{{.Size}}"
```

output:
```shell
130MB # Q5
```

***This cell is running in the GitHub Codespace environment.***

```shell
docker build -t subscription-prediction .
docker run -it -p 9696:9696 subscription-prediction:latest
```

In [20]:
url = "https://cold-cadaver-7vrrxw5j5x763r5g4-9696.app.github.dev/predict" # copied from GitHub Codespace's session
client = {"job": "management", "duration": 400, "poutcome": "success"}
requests.post(url, json=client).json() # Q6

{'subscription': True, 'subscription_probability': 0.7590966516879658}